## Imports

In [ ]:
import sys, os, json, re
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.scraper import get_recipe_data
from src.ingredients_parser import IngredientsParser
from src.methods_parser import MethodsParser
from src.tools_parser import ToolsParser
from src.steps_parser import StepsParser

with open("../src/helper_files/ground_truth.json", "r", encoding="utf-8") as f:
    ground_truth = json.load(f)

allrecipes_urls = [
    "https://www.allrecipes.com/recipe/21014/good-old-fashioned-pancakes/",
    "https://www.allrecipes.com/recipe/24074/alysias-basic-meat-lasagna/",
    "https://www.allrecipes.com/recipe/228293/curry-stand-chicken-tikka-masala-sauce/",
    "https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/",
    "https://www.allrecipes.com/recipe/238543/grandmas-cucumber-and-onion-salad/",
    "https://www.allrecipes.com/recipe/15375/fried-chicken-with-creamy-gravy/",
    "https://www.allrecipes.com/recipe/218288/garlic-spinach/",
    "https://www.allrecipes.com/recipe/57348/balsamic-glazed-carrots/",
    "https://www.allrecipes.com/recipe/254558/refreshing-oatmeal-drink-agua-de-avena/",
    "https://www.allrecipes.com/recipe/202975/potstickers-chinese-dumplings/"
]
epicurious_urls = [
    "https://www.epicurious.com/recipes/food/views/easy-fried-rice",
    "https://www.epicurious.com/recipes/food/views/ba-syn-easy-apple-cake",
    "https://www.epicurious.com/recipes/food/views/ba-syn-spicy-cashew-scallion-noodles",
    "https://www.epicurious.com/recipes/food/views/ba-syn-sweet-and-sour-cranberry-chicken-stir-fry",
    "https://www.epicurious.com/recipes/food/views/ba-syn-peach-blueberry-pie",
    "https://www.epicurious.com/recipes/food/views/tomato-galette",
    "https://www.epicurious.com/recipes/food/views/ba-syn-ginger-cardamom-zucchini-bread",
    "https://www.epicurious.com/recipes/food/views/white-bean-turkey-chili",
    "https://www.epicurious.com/recipes/food/views/thai-curry-puff-ga-ree-puff",
    "https://www.epicurious.com/recipes/food/views/steamed-winter-veggie-bowls"
]
bonappetit_urls = [
    "https://www.bonappetit.com/recipe/kale-pesto-with-whole-wheat-pasta",
    "https://www.bonappetit.com/recipe/adult-spaghettios",
    "https://www.bonappetit.com/recipe/ham-cheese-and-onion-empanadas",
    "https://www.bonappetit.com/recipe/crispy-smashed-potatoes-with-walnut-dressing",
    "https://www.bonappetit.com/recipe/zucchini-lentil-fritters-with-lemony-yogurt",
    "https://www.bonappetit.com/recipe/grilled-chicken-skewers-with-toum-shish-taouk",
    "https://www.bonappetit.com/recipe/bas-best-apple-pie",
    "https://www.bonappetit.com/recipe/ba-best-lasagna",
    "https://www.bonappetit.com/recipe/bas-best-chocolate-chip-cookies",
    "https://www.bonappetit.com/recipe/spicy-salmon-bowl"
]

allrecipes_data = allrecipes_urls.copy() + epicurious_urls.copy() + bonappetit_urls.copy()

## Testing Ingredients Parser

In [ ]:
parsed_components = []
for i in range(len(allrecipes_data)):
    title, ingredients, directions = get_recipe_data(allrecipes_data[i])
    ingreadient_parser = IngredientsParser(ingredients)
    parsed_components.append(ingreadient_parser.parse())

In [ ]:
def show_differences(ground_truth, parsed):
    """
    Shows only mismatched ingredients, with normalization-aware comparison,
    but prints ONLY the raw GT and Parsed values (no normalized echoes).
    """

    RED   = "\033[91m"
    GREEN = "\033[92m"
    CYAN  = "\033[96m"
    YELLOW= "\033[93m"
    RESET = "\033[0m"
    BOLD  = "\033[1m"

    import re

    NULL_LIKE = {"", "null", "none", "NaN", "nan", "NULL", "None"}

    # unicode vulgar fraction map
    VF = {
        "¼": 0.25, "½": 0.5, "¾": 0.75,
        "⅐": 1/7, "⅑": 1/9, "⅒": 0.1,
        "⅓": 1/3, "⅔": 2/3,
        "⅕": 0.2, "⅖": 0.4, "⅗": 0.6, "⅘": 0.8,
        "⅙": 1/6, "⅚": 5/6,
        "⅛": 0.125,"⅜": 0.375,"⅝": 0.625,"⅞": 0.875
    }

    def as_none(x):
        if x is None:
            return None
        if isinstance(x, str) and x.strip() in NULL_LIKE:
            return None
        return x

    def norm_text(x):
        x = as_none(x)
        if x is None:
            return None
        return re.sub(r"\s+", " ", str(x).strip()).lower() or None

    def parse_num(x):
        x = as_none(x)
        if x is None:
            return None
        if isinstance(x, (int, float)):
            return float(x)

        s = str(x).strip()

        # unicode fraction
        for f, val in VF.items():
            if f in s:
                parts = s.split(f)
                if parts[0].strip().isdigit():
                    return float(parts[0]) + val
                return val

        # mixed fraction (e.g., 1 1/2)
        m = re.match(r"^(\d+)\s+(\d+)\/(\d+)$", s)
        if m:
            whole, num, den = map(int, m.groups())
            return whole + num/den

        # simple fraction (1/2)
        m = re.match(r"^(\d+)\/(\d+)$", s)
        if m:
            num, den = map(int, m.groups())
            return num/den

        # plain decimal/int
        try:
            return float(s)
        except:
            return None

    def same_name(a, b):
        return norm_text(a) == norm_text(b)

    def same_qty(a, b):
        na, nb = parse_num(a), parse_num(b)
        if na is None and nb is None:
            return True
        if na is None or nb is None:
            return False
        return abs(na - nb) < 1e-9

    def same_unit(a, b):
        ua, ub = norm_text(a), norm_text(b)
        if ua == ub:
            return True
        # treat trivial plural differences as equal
        if ua and ub and ua.rstrip("s") == ub.rstrip("s"):
            return True
        return False

    def norm_list(xs):
        if xs is None:
            return set()
        if isinstance(xs, str):
            xs = [xs]
        cleaned = {norm_text(x) for x in xs if norm_text(x)}
        return cleaned

    def same_list(a, b):
        return norm_list(a) == norm_list(b)

    # ---------------- PRINT DIFFS ----------------
    any_diff = False

    for idx, (gt, pr) in enumerate(zip(ground_truth, parsed), start=1):
        diffs = []

        if not same_name(gt.get("ingredient_name"), pr.get("ingredient_name")):
            diffs.append(( "Name", gt.get("ingredient_name"), pr.get("ingredient_name") ))

        if not same_qty(gt.get("ingredient_quantity"), pr.get("ingredient_quantity")):
            diffs.append(( "Quantity", gt.get("ingredient_quantity"), pr.get("ingredient_quantity") ))

        if not same_unit(gt.get("measurement_unit"), pr.get("measurement_unit")):
            diffs.append(( "Unit", gt.get("measurement_unit"), pr.get("measurement_unit") ))

        if not same_list(gt.get("ingredient_descriptors"), pr.get("ingredient_descriptors")):
            diffs.append(( "Descriptors", gt.get("ingredient_descriptors"), pr.get("ingredient_descriptors") ))

        if not same_list(gt.get("ingredient_preparation"), pr.get("ingredient_preparation")):
            diffs.append(( "Preparation", gt.get("ingredient_preparation"), pr.get("ingredient_preparation") ))

        if diffs:
            any_diff = True
            print(f"\n{BOLD}Ingredient #{idx}{RESET}  {CYAN}{gt.get('original_ingredient_sentence', '')}{RESET}")
            print("─" * 80)
            for label, g, p in diffs:
                print(f"{YELLOW}{label:<12}{RESET}{CYAN}{g!r}{RESET}  →  {RED}{p!r}{RESET}")

    if not any_diff:
        print(f"{GREEN}No differences found.{RESET}\n")
def show_differences_names(ground_truth, parsed):
    """
    Shows only mismatched ingredient names.
    """

    RED   = "\033[91m"
    GREEN = "\033[92m"
    CYAN  = "\033[96m"
    YELLOW= "\033[93m"
    RESET = "\033[0m"
    BOLD  = "\033[1m"
    print(f"{BOLD}Total ingredients: {len(ground_truth)}{RESET}")


    NULL_LIKE = {"", "null", "none", "NaN", "nan", "NULL", "None"}

    def as_none(x):
        if x is None:
            return None
        if isinstance(x, str) and x.strip() in NULL_LIKE:
            return None
        return x

    def norm_text(x):
        x = as_none(x)
        if x is None:
            return None
        return re.sub(r"\s+", " ", str(x).strip()).lower() or None

    def same_name(a, b):
        return norm_text(a) == norm_text(b)

    any_diff = False

    for idx, (gt, pr) in enumerate(zip(ground_truth, parsed), start=1):
        gt_name = gt.get("ingredient_name")
        pr_name = pr.get("ingredient_name")

        if not same_name(gt_name, pr_name):
            any_diff = True
            print(f"\n{BOLD}Ingredient #{idx}{RESET}  {CYAN}{gt.get('original_ingredient_sentence', '')}{RESET}")
            print("─" * 80)
            print(f"{YELLOW}Name:{RESET} {CYAN}{gt_name!r}{RESET}  →  {RED}{pr_name!r}{RESET}")

    if not any_diff:
        print(f"{GREEN}No name differences found.{RESET}\n")
def performance(ground_truth, parsed_components):

    # --- helpers (local-only; no printing) ---
    def _is_none_like(x):
        if x is None:
            return True
        if isinstance(x, str) and x.strip().lower() in {"", "none", "null", "n/a", "na"}:
            return True
        return False

    def _norm_spaces(s: str) -> str:
        return re.sub(r"\s+", " ", s).strip()

    def _norm_name(x):
        if _is_none_like(x):
            return None
        return _norm_spaces(str(x)).lower()

    # Handle numbers that may come as int/float/str (including simple fractions)
    _vulgar_map = {
        "¼": 0.25, "½": 0.5, "¾": 0.75,
        "⅐": 1/7, "⅑": 1/9, "⅒": 0.1,
        "⅓": 1/3, "⅔": 2/3, "⅕": 0.2, "⅖": 0.4, "⅗": 0.6, "⅘": 0.8,
        "⅙": 1/6, "⅚": 5/6, "⅛": 0.125, "⅜": 0.375, "⅝": 0.625, "⅞": 0.875,
    }

    def _to_number(x):
        if _is_none_like(x):
            return None
        if isinstance(x, (int, float)):
            return float(x)
        s = str(x).strip()
        # whole + vulgar, e.g., "1 ½"
        m = re.match(r"^\s*(\d+)\s*([{}])\s*$".format("".join(map(re.escape, _vulgar_map.keys()))), s)
        if m:
            return float(m.group(1)) + float(_vulgar_map.get(m.group(2), 0.0))
        # vulgar only, e.g., "½"
        if s in _vulgar_map:
            return float(_vulgar_map[s])
        # a/b or whole a/b, e.g., "1/2", "1 1/2"
        m = re.match(r"^\s*(?:(\d+)\s+)?(\d+)\s*/\s*(\d+)\s*$", s)
        if m:
            whole = float(m.group(1)) if m.group(1) else 0.0
            num = int(m.group(2)); den = int(m.group(3)) or 1
            return whole + (num / den)
        # simple float/int
        m = re.match(r"^\s*-?\d+(?:\.\d+)?\s*$", s)
        if m:
            return float(s)
        # otherwise unparseable -> None
        return None

    def _numbers_equal(a, b):
        na, nb = _to_number(a), _to_number(b)
        if na is None and nb is None:
            return True
        if (na is None) != (nb is None):
            return False
        # compare with small tolerance
        return abs(na - nb) <= 1e-9

    def _norm_unit(u):
        if _is_none_like(u):
            return None
        s = _norm_spaces(str(u)).lower().rstrip(".")
        # tolerate trivial plural/singular differences (cup/cups, ounce/ounces, lb/lbs)
        s = re.sub(r"s\b", "", s)  # cheap plural trim (safe for our canonical set)
        return s

    def _units_equal(a, b):
        ua, ub = _norm_unit(a), _norm_unit(b)
        return ua == ub

    def _norm_list(xs):
        if xs is None:
            return set()
        if isinstance(xs, str):
            xs = [xs]
        try:
            normed = []
            for x in xs:
                if _is_none_like(x):
                    continue
                t = _norm_spaces(str(x)).lower().strip(",.;:")
                if t:
                    normed.append(t)
            return set(normed)
        except TypeError:
            # Not iterable: coerce to single item
            return _norm_list([xs])

    # --- metrics ---
    ing_name_correct = 0
    ing_quantity_correct = 0
    ing_unit_correct = 0
    total_ingredients = len(ground_truth)

    ing_prep_correct = 0
    total_preps = 0

    ing_descriptors_correct = 0
    total_descriptors = 0

    for gt, ping in zip(ground_truth, parsed_components):
        # Ingredient-level accuracy
        if _norm_name(gt.get('ingredient_name')) == _norm_name(ping.get('ingredient_name')):
            ing_name_correct += 1

        if _numbers_equal(gt.get('ingredient_quantity'), ping.get('ingredient_quantity')):
            ing_quantity_correct += 1

        if _units_equal(gt.get('measurement_unit'), ping.get('measurement_unit')):
            ing_unit_correct += 1

        # --- Descriptor Scoring ---
        gt_desc = _norm_list(gt.get('ingredient_descriptors'))
        p_desc = _norm_list(ping.get('ingredient_descriptors'))

        if len(gt_desc) == 0 and len(p_desc) == 0:
            total_descriptors += 1
            ing_descriptors_correct += 1
        else:
            total_descriptors += len(gt_desc)
            ing_descriptors_correct += len(gt_desc & p_desc)

        # --- Preparation Scoring ---
        gt_prep = _norm_list(gt.get('ingredient_preparation'))
        p_prep = _norm_list(ping.get('ingredient_preparation'))

        if len(gt_prep) == 0 and len(p_prep) == 0:
            total_preps += 1
            ing_prep_correct += 1
        else:
            total_preps += len(gt_prep)
            ing_prep_correct += len(gt_prep & p_prep)

    return {
        "name_accuracy": ing_name_correct / total_ingredients if total_ingredients else 1,
        "quantity_accuracy": ing_quantity_correct / total_ingredients if total_ingredients else 1,
        "unit_accuracy": ing_unit_correct / total_ingredients if total_ingredients else 1,
        "descriptor_accuracy": ing_descriptors_correct / total_descriptors if total_descriptors > 0 else 1,
        "preparation_accuracy": ing_prep_correct / total_preps if total_preps > 0 else 1
    }

In [ ]:
all_scores = {
    "name_accuracy": 0,
    "quantity_accuracy": 0,
    "unit_accuracy": 0,
    "descriptor_accuracy": 0,
    "preparation_accuracy": 0
}

for idx in range(len(ground_truth)):
    result = performance(ground_truth[idx], parsed_components[idx])
    
    for key in all_scores:
        all_scores[key] += result[key]

for key in all_scores:
    all_scores[key] /= len(ground_truth)

all_scores

for i in range(len(ground_truth)):
    show_differences_names(ground_truth[i], parsed_components[i])

In [ ]:
title, ingredients, directions = get_recipe_data(allrecipes_data[1])
ingreadient_parser = IngredientsParser(ingredients)
ing = ingreadient_parser.parse()


## Testing Methods and Tools Parsers

In [ ]:
title, ingredients, directions = get_recipe_data(allrecipes_data[2])

method_parser = MethodsParser(directions)
methods = method_parser.parse()

tool_parser = ToolsParser(directions)
tools = tool_parser.parse()

for i, step in enumerate(directions['directions']):
    print(f"step_{i+1}: {step}")
    print(f"methods: {methods[i]['methods']}")
    print(f"tools: {tools[i]['tools']}")
    print()

## Testing Steps Parser

In [ ]:
title, ingredients, directions = get_recipe_data(allrecipes_data[1])
steps_parser = StepsParser(directions, IngredientsParser(ingredients).parse())
steps = steps_parser.parse()

In [ ]:
from pprint import pprint
sep = "─" * 60

for step in steps:
    print(f"\n{sep}")
    print(f"Step {step['step_number']}: {step['description']}")
    print(f"{sep}")

    def fmt(label, value):
        if not value:
            return f"  {label}: —"
        if isinstance(value, list):
            return f"  {label}: {', '.join(value)}"
        return f"  {label}: {value}"

    print(fmt("Ingredients", step.get("ingredients")))
    print(fmt("Tools", step.get("tools")))
    print(fmt("Methods", step.get("methods")))
    print(fmt("Time", step.get("time")))
    print(fmt("Temperature", step.get("temperature")))
    print(fmt("Type", step.get("type")))

print(f"\n{sep}\nDone.\n{sep}")

In [ ]:
for c in IngredientsParser(ingredients).parse():
    print(c["ingredient_name"])